In [2]:
import requests  
from bs4 import BeautifulSoup #
import json
import urllib
import time   
import re
import urlparse
import urllib2

#第一步，构建书籍的id列表

keyword = 'python' #关键词(可以修改)
page = 0###################第几页  ###可以循环N页（奇数）
book_num = 100 #一共100本书
list_jd = [[] for i in range(book_num)]

#首先抓取第1页前30个结果的id
params = {'keyword':keyword,'enc':'utf-8','qrst':1,'rt':1,'stop':1,'vt':2,'page':page,'click':0}
data = urllib.urlencode(params)
opener = urllib2.urlopen("https://search.jd.com/Search?"+data)
content = opener.read()
opener.close()
soup = BeautifulSoup(content,"html.parser")
goods_info = soup.select(".gl-item")
pid = []
for good_info in goods_info:
    pid.append(good_info.attrs['data-sku'])

#再抓取第1页后30个结果的id
page=page*2+1
other_url = "https://search.jd.com/s_new.php?keyword="+keyword+"&enc=utf-8&qrst=1&rt=1&stop=1&vt=2&wq="+keyword+"&page="+str(page+1)+"&scrolling=y&log_id={}&tpl=2_M&show_items={}"###s需要修正，往往不定值
linux_sec = str(time.time())[:-2]#时间戳去掉后两位
other_url = other_url.format(linux_sec, ','.join(pid))#构造后半部分网页，有三处要除以一个page+1，一处是log_id为时间戳去掉后两位，最后的一堆列表是前半部分解析的data-pid构成的
headers = {
    'referer': "https://search.jd.com/Search?{}".format(str(page))
}
r = requests.get(url = other_url, headers=headers,verify = False,timeout=15)
r.encoding = r.apparent_encoding
response = r.text
for i in range(30):
    pida = response.split('data-sku=')[i*2+1].split('\"')[1]
    pid.append(pida)
    
#抓取第2页前30个结果的id
page = page*2+1
params = {'keyword':keyword,'enc':'utf-8','qrst':1,'rt':1,'stop':1,'vt':2,'page':page,'click':0}
data = urllib.urlencode(params)
opener = urllib2.urlopen("https://search.jd.com/Search?"+data)
content = opener.read()
opener.close()
soup = BeautifulSoup(content,"html.parser")
goods_info = soup.select(".gl-item")
for good_info in goods_info:
    pid.append(good_info.attrs['data-sku'])

#抓取第2页后30个结果的id
other_url = "https://search.jd.com/s_new.php?keyword="+keyword+"&enc=utf-8&qrst=1&rt=1&stop=1&vt=2&wq="+keyword+"&page="+str(page+1)+"&scrolling=y&log_id={}&tpl=2_M&show_items={}"###s需要修正，往往不定值
linux_sec = str(time.time())[:-2]
other_url = other_url.format(linux_sec, ','.join(pid))
headers = {
    'referer': "https://search.jd.com/Search?{}".format(str(page))
}
r = requests.get(url = other_url, headers=headers,verify = False,timeout=15)
r.encoding = r.apparent_encoding
response = r.text
for i in range(30):
    pida = response.split('data-sku=')[i*2+1].split('\"')[1]
    pid.append(pida)

#print pid    
#发现id存在重复，进行去重,同时保证id顺序按照搜索结果排序
pid_ =list(set(pid))
pid_.sort(key=pid.index)
print pid_,len(pid_)


C:\Users\Kirihara\Anaconda3\envs\py27\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\Kirihara\Anaconda3\envs\py27\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[u'12458274', u'11993134', u'12353915', u'12451724', u'11936238', u'12279949', u'12398725', u'12445029', u'12372646', u'11598704', u'11943853', u'12335366', u'12333540', u'12417265', u'12468732', u'12392747', u'12186192', u'11821937', u'12273412', u'12409581', u'12004711', u'30201551500', u'10599758', u'12273591', u'12461168', u'11681561', u'11352441', u'11841674', u'11487324', u'11848567', u'27176955108', u'12452929', u'14017202143', u'12333252', u'12364204', u'11576833', u'11896415', u'11896401', u'26809408972', u'12428772', u'12366000', u'12293703', u'12375644', u'12397576', u'12292223', u'12473144', u'12403048', u'12451736', u'12280691', u'12230702', u'11996409', u'26147900526', u'12125529', u'12148832', u'11864820', u'12028953', u'12425594', u'11967520', u'12477870', u'10062788', u'14017404441', u'12407729', u'12750925326', u'12442779', u'11748995', u'12186435', u'12405421', u'11666319', u'11918059', u'11485222', u'12219342', u'11571426', u'12460540', u'12193983', u'11593982', u'1

In [4]:
import requests  #
from bs4 import BeautifulSoup #
import json
import urllib
import numpy as np
import random

book_num = 100 #一共100本书
list_jd = [[] for i in range(book_num)]
t = 3 #被截断后休息的最长时间
test = True #显示测试过程
count = 0 #计数
for i in range(book_num):
    count +=1
    
    if test:
        print 'Getting book ' + str(count) + ' , with ' + str(book_num) + ' books in total' #显示抓取进度
    
    sku = int(pid_[i])
    url = 'https://item.jd.com/' + bytes(sku) + '.html'  
    html_ = requests.get(url).content
    soup_page = BeautifulSoup(html_,'html5lib') 
    books_name = soup_page.find('div',class_="sku-name").strings
    for str_ in books_name:
        name_ = str_.strip()
    #print name_
    list_jd[i].append(name_)

    price_url = "https://p.3.cn/prices/mgets?skuIds=J_" + bytes(sku)
    response = urllib.urlopen(price_url) 
    content = response.read() 
    content = content.decode("utf-8")
    result = json.loads(content) 
    record = result[0] 
    price=record['p'] 
    list_jd[i].append(price)

    comment_ = []
    for j in range(1,100): #京东商品页面最多显示100页评论
        url_comm = 'https://sclub.jd.com/comment/productPageComments.action?productId=' + bytes(sku) + '&score=0&sortType=5&'+'page='+str(j)+'&pageSize=10&isShadowSku=0&rid=0&fold=1'
        
        try:#异常抛出
            html_comm=requests.get(url_comm).text
            if html_comm:
                json_html_comm=json.loads(html_comm)
                comment_list=json_html_comm['comments']
            else:
                break
        except:#被截断访问后休眠1~3秒后继续访问
            time.sleep(random.randint(1,t))
            html_comm=requests.get(url_comm).text
            if html_comm:
                json_html_comm=json.loads(html_comm)
                ocomment_list=json_html_comm['comments']
            else:
                break
        for se in comment_list:
            comment_.append(se['content'])        
        #剔除重复的评论,保留按页面展示结果的排序
        comment_1 = list(set(comment_))
        comment_1.sort(key=comment_.index)
        
    book_comment= '####'.join(comment_1)
    list_jd[i].append(book_comment)

print list_jd


Getting book 1 , with 100 books in total
Getting book 2 , with 100 books in total
Getting book 3 , with 100 books in total
Getting book 4 , with 100 books in total
Getting book 5 , with 100 books in total
Getting book 6 , with 100 books in total
Getting book 7 , with 100 books in total
Getting book 8 , with 100 books in total
Getting book 9 , with 100 books in total
Getting book 10 , with 100 books in total
Getting book 11 , with 100 books in total
Getting book 12 , with 100 books in total
Getting book 13 , with 100 books in total
Getting book 14 , with 100 books in total
Getting book 15 , with 100 books in total
Getting book 16 , with 100 books in total
Getting book 17 , with 100 books in total
Getting book 18 , with 100 books in total
Getting book 19 , with 100 books in total
Getting book 20 , with 100 books in total
Getting book 21 , with 100 books in total
Getting book 22 , with 100 books in total
Getting book 23 , with 100 books in total
Getting book 24 , with 100 books in total
G

In [6]:
import pandas as pd
columns=['书名','售价','评论']
books = pd.DataFrame(list_jd,columns=columns)
books.to_csv("Jingdong.csv",index=False,encoding="utf-8")
print books

                                                   书名      售价  \
0                            Python编程从零基础到项目实战（微课视频版）   66.20   
1                                     Python编程 从入门到实践   72.70   
2                                     零基础学Python（全彩版）   57.90   
3                                 Python从入门到项目实践（全彩版）   72.40   
4                                                       82.90   
5                                     Python基础教程（第3版）   80.90   
6                                                       95.80   
7                                      Python编程从入门到精通   66.20   
8                                        笨办法学Python 3   49.40   
9                                 Python编程（第4版 套装上下册）  157.00   
10                              Python编程快速上手 让繁琐工作自动化   57.80   
11                                       Python神经网络编程   57.80   
12                                   Python 3网络爬虫开发实战   80.90   
13                                       Python从菜鸟到高手  106.10   
14                       